# Fog Classification Training Pipeline

This notebook trains a ResNet-50 classifier for fog detection using FADE-based weak labels.

**Dataset Split**: 80% Train, 20% Test

**Classes**: 
- Clear (score < 1.0)
- Light Fog (1.0 ≤ score < 2.0)  
- Dense Fog (score ≥ 2.0)

## 1. Import Required Libraries

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
from torchvision import transforms
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from cnnClassifier import FogResNet50Classifier

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.1
CUDA available: False


## 2. Define Dataset Class

In [31]:
class FogDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe: pandas DataFrame with columns ['image_id', 'image_path', 'score', 'weak_label']
            transform: torchvision transforms
        """
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        
        # Map labels to integers
        self.label_map = {'clear': 0, 'light_fog': 1, 'dense_fog': 2}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Use absolute path from parquet file
        image_path = row['image_path']
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {image_path}: {e}")
            # Return a black image as fallback
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        # Get label and normalized score (normalize to 0-1 range for regression head)
        label = self.label_map[row['weak_label']]
        # Normalize score: cap at max 3.0 and divide by 3
        normalized_score = min(float(row['score']), 3.0) / 3.0
        
        return image, label, normalized_score

## 3. Define Data Transforms

In [32]:
def get_transforms(is_training=True):
    """Get data augmentation transforms"""
    if is_training:
        return transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])

## 4. Helper Functions

In [33]:
def create_weak_labels(df):
    """
    Create weak labels from FADE scores.
    Thresholds: clear (0-1), light_fog (1-2), dense_fog (>2)
    """
    if 'weak_label' in df.columns:
        print("Weak labels already exist in dataframe")
        return df
    
    if 'score' not in df.columns:
        raise ValueError("DataFrame must contain 'score' column")
    
    print("Creating weak labels from FADE scores...")
    print(f"Score range: {df['score'].min():.2f} - {df['score'].max():.2f}")
    
    def score_to_label(score):
        if score < 1.0:
            return 'clear'
        elif score < 2.0:
            return 'light_fog'
        else:
            return 'dense_fog'
    
    df['weak_label'] = df['score'].apply(score_to_label)
    
    print("\nWeak labels created:")
    print(df['weak_label'].value_counts())
    print("\nScore distribution per label:")
    for label in ['clear', 'light_fog', 'dense_fog']:
        if label in df['weak_label'].values:
            scores = df[df['weak_label'] == label]['score']
            print(f"{label}: mean={scores.mean():.2f}, min={scores.min():.2f}, max={scores.max():.2f}")
    
    return df


def calculate_class_weights(df):
    """Calculate class weights for handling imbalance"""
    label_counts = df['weak_label'].value_counts()
    total = len(df)
    
    weights = {}
    for label in ['clear', 'light_fog', 'dense_fog']:
        if label in label_counts:
            weights[label] = total / (len(label_counts) * label_counts[label])
        else:
            weights[label] = 1.0
    
    # Convert to tensor in order [clear, light_fog, dense_fog]
    weight_tensor = torch.tensor([weights['clear'], weights['light_fog'], weights['dense_fog']], 
                                dtype=torch.float32)
    
    print(f"Class distribution:")
    print(label_counts)
    print(f"\nClass weights: {weight_tensor}")
    
    return weight_tensor

## 5. Training and Validation Functions

In [34]:
def train_epoch(model, train_loader, class_criterion, density_criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, labels, fade_scores in pbar:
        images = images.to(device)
        labels = labels.to(device)
        fade_scores = fade_scores.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        
        # Forward pass
        class_logits, density_pred = model(images)
        
        # Calculate losses
        class_loss = class_criterion(class_logits, labels)
        density_loss = density_criterion(density_pred, fade_scores)
        loss = class_loss + 0.5 * density_loss  # Combined loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = torch.max(class_logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100*correct/total:.2f}%'})
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy


def validate(model, test_loader, class_criterion, density_criterion, device):
    """Validate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    all_densities = []
    all_fade_scores = []
    
    with torch.no_grad():
        for images, labels, fade_scores in tqdm(test_loader, desc='Testing'):
            images = images.to(device)
            labels = labels.to(device)
            fade_scores = fade_scores.to(device).float().unsqueeze(1)
            
            class_logits, density_pred = model(images)
            
            class_loss = class_criterion(class_logits, labels)
            density_loss = density_criterion(density_pred, fade_scores)
            loss = class_loss + 0.5 * density_loss
            
            total_loss += loss.item()
            _, predicted = torch.max(class_logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Store predictions for analysis
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_densities.extend(density_pred.cpu().numpy())
            all_fade_scores.extend(fade_scores.cpu().numpy())
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    # Calculate correlation between predicted density and FADE scores
    correlation = np.corrcoef(np.array(all_densities).flatten(), 
                             np.array(all_fade_scores).flatten())[0, 1]
    
    return avg_loss, accuracy, correlation

## 6. Configuration and Setup

In [35]:
# Configuration
PARQUET_FILE = '../data/fade_results_complete.parquet'
MODELS_FOLDER = '../models'

BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
NUM_WORKERS = 0  # Set to 0 for notebook compatibility (multiprocessing doesn't work in notebooks)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create models folder if it doesn't exist
os.makedirs(MODELS_FOLDER, exist_ok=True)

print(f"Using device: {DEVICE}")
print(f"Models will be saved to: {MODELS_FOLDER}")

Using device: cpu
Models will be saved to: ../models


## 7. Load and Prepare Data

In [36]:
# Load data
print(f"Loading data from {PARQUET_FILE}...")
df = pd.read_parquet(PARQUET_FILE)
print(f"Loaded {len(df)} samples")
print(f"Columns: {df.columns.tolist()}")

# Show first few rows and statistics
print("\nFirst few rows:")
display(df.head())

print(f"\nScore statistics:")
display(df['score'].describe())

Loading data from ../data/fade_results_complete.parquet...
Loaded 19811 samples
Columns: ['image_id', 'image_path', 'score']

First few rows:


,image_id,image_path,score
0,foggy_012970.jpg,/Users/deb/Documents/projects/research-project...,0.889074
1,foggy_006905.jpg,/Users/deb/Documents/projects/research-project...,0.423859
2,foggy_019825.jpg,/Users/deb/Documents/projects/research-project...,3.375630
3,foggy_017808.jpg,/Users/deb/Documents/projects/research-project...,1.053913
4,foggy_010801.jpg,/Users/deb/Documents/projects/research-project...,3.503386



Score statistics:


count    19811.000000
mean         2.752235
std          2.031054
min          0.072503
25%          1.136188
50%          2.200730
75%          3.851727
max         12.177445
Name: score, dtype: float64

In [37]:
# Create weak labels
df = create_weak_labels(df)

Creating weak labels from FADE scores...
Score range: 0.07 - 12.18

Weak labels created:
weak_label
dense_fog    10646
light_fog     5098
clear         4067
Name: count, dtype: int64

Score distribution per label:
clear: mean=0.70, min=0.07, max=1.00
light_fog: mean=1.45, min=1.00, max=2.00
dense_fog: mean=4.16, min=2.00, max=12.18


In [38]:
# Split data into train (80%) and test (20%)
min_samples_per_class = df['weak_label'].value_counts().min()

if len(df) < 10:
    print(f"\nWARNING: Very small dataset ({len(df)} samples)!")
    print("Using all data for both training and testing (no split).")
    train_df = df.copy()
    test_df = df.copy()
elif min_samples_per_class < 2:
    print(f"\nWARNING: Some classes have fewer than 2 samples!")
    print("Splitting without stratification...")
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
else:
    # Normal split with stratification (80% train, 20% test)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, 
                                        stratify=df['weak_label'])

print(f"\nTrain samples: {len(train_df)} (80%)")
print(f"Test samples: {len(test_df)} (20%)")


Train samples: 15848 (80%)
Test samples: 3963 (20%)


## 8. Create Datasets and Data Loaders

In [39]:
# Calculate class weights for imbalance handling
class_weights = calculate_class_weights(train_df).to(DEVICE)

# Create datasets (image paths are already in the dataframe)
train_dataset = FogDataset(train_df, transform=get_transforms(is_training=True))
test_dataset = FogDataset(test_df, transform=get_transforms(is_training=False))

# Create data loaders (num_workers=0 for notebook compatibility)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                        shuffle=True, num_workers=NUM_WORKERS, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, 
                      shuffle=False, num_workers=NUM_WORKERS, pin_memory=False)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

Class distribution:
weak_label
dense_fog    8516
light_fog    4078
clear        3254
Name: count, dtype: int64

Class weights: tensor([1.6234, 1.2954, 0.6203])

Train batches: 496
Test batches: 124


## 9. Initialize Model and Training Components

In [40]:
# Initialize model
print("Initializing ResNet-50 model...")
model = FogResNet50Classifier(num_classes=3, pretrained=True).to(DEVICE)

# Loss functions
class_criterion = nn.CrossEntropyLoss(weight=class_weights)
density_criterion = nn.MSELoss()

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

print(f"✓ Model initialized with {sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

Initializing ResNet-50 model...
✓ Model initialized with 25.08M parameters


/Users/deb/Documents/projects/research-project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/deb/Documents/projects/research-project/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 10. Training Loop

In [41]:
best_test_acc = 0

print("Starting training...\n")

for epoch in range(NUM_EPOCHS):
    print(f"{'='*50}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print('='*50)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, class_criterion, 
                                       density_criterion, optimizer, DEVICE)
    
    # Test
    test_loss, test_acc, correlation = validate(model, test_loader, class_criterion, 
                                              density_criterion, DEVICE)
    
    # Update learning rate
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    print(f"\nResults:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print(f"  Density Correlation: {correlation:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")
    
    # Save best model
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        model_path = os.path.join(MODELS_FOLDER, 'best_fog_resnet50.pth')
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'test_acc': test_acc,
            'test_loss': test_loss,
            'correlation': correlation,
        }, model_path)
        
        print(f"  ✓ Saved best model (Test Acc: {test_acc:.2f}%)")
    
    print()

print("="*50)
print("Training completed!")
print(f"Best test accuracy: {best_test_acc:.2f}%")
print(f"Model saved at: {os.path.join(MODELS_FOLDER, 'best_fog_resnet50.pth')}")
print("="*50)

Starting training...

Epoch 1/20


Testing: 100%|██████████| 124/124 [04:10<00:00,  2.02s/it]



Results:
  Train Loss: 0.4695, Train Acc: 81.85%
  Test Loss: 0.3102, Test Acc: 88.04%
  Density Correlation: 0.9680
  Learning Rate: 0.000099
  ✓ Saved best model (Test Acc: 88.04%)

Epoch 2/20


Testing: 100%|██████████| 124/124 [04:10<00:00,  2.02s/it]



Results:
  Train Loss: 0.3921, Train Acc: 84.48%
  Test Loss: 0.3363, Test Acc: 85.47%
  Density Correlation: 0.9710
  Learning Rate: 0.000098

Epoch 3/20


...


Results:
  Train Loss: 0.2591, Train Acc: 90.24%
  Test Loss: 0.3440, Test Acc: 87.81%
  Density Correlation: 0.9721
  Learning Rate: 0.000065

Epoch 9/20


Testing: 100%|██████████| 124/124 [04:22<00:00,  2.12s/it]



Results:
  Train Loss: 0.2391, Train Acc: 91.24%
  Test Loss: 0.3020, Test Acc: 87.48%
  Density Correlation: 0.9742
  Learning Rate: 0.000058

Epoch 10/20


Training:   0%|          | 1/496 [00:07<59:02,  7.16s/it, loss=0.3066, acc=90.62%]


KeyboardInterrupt: 